# Automatic Image Colorization

Based on [Let there be Color!](http://hi.cs.waseda.ac.jp/~iizuka/projects/colorization/en/)

In [ ]:
import os
import time

import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

from skimage import color, io
import matplotlib.pyplot as plt


torch.set_default_tensor_type('torch.FloatTensor')

### Hyperparameters

In [ ]:
BATCH_SIZE = 4
EPOCHS = 30
img_dir_train = './data/food41-120-train/'
img_dir_test  = './data/food41-120-train/' #test/'
img_dir_dev   = './data/food41-120-test/'

# TODO

- [x] convert dataset to tensor? (now: numpy array)
 - seems like dataloader does it automatically - but doesnt swap axes
 - transforms.toTensor does the job
- [x] normialize data?
 - only $ab$ channel + L channel
- [ ] transform images? - random crops itp
- [ ] deal with `torch.set_default_tensor_type('torch.DoubleTensor')` or `torch.FloatTensor`
- [x] refector asserts so they work with batch sizes different than 4
- [ ] shuffle data (trainloader)
- [ ] refactor?? - separate files
- [x] load / save model
- [x] extract hyperparameters
- [x] load whole dataset to memory
- [x] read/write to zip
- [ ] deal with `Color data out of range: Z < 0 in %s pixels`

### Custom dateset

- One needs to [swap axes](https://pytorch.org/tutorials/beginner/data_loading_tutorial.html#transforms)
    - numpy image: H x W x C
    - torch image: C X H X W


- Quick fact: $L \in [0, 100]$, $a \in [-127, 128]$, $b \in [-128, 127]$ [(source)](https://stackoverflow.com/questions/25294141/cielab-color-range-for-scikit-image)


- How an image is processed:
    1. Load i-th image to memory
    2. Convert it to LAB Space
    3. Convert to torch.tensor
    4. Split the image to $L$ and $ab$ channels
    5. Normalize $ab$ to $[0, 1]$. $L$ ~~remains unnormalized.~~ is normalized as well.
    6. $L$ will feed net, $a'b'$ will be its output
    7. Calculate $Loss(ab, a'b')$

In [ ]:
class ImagesDateset(Dataset):
    
    def __init__(self, img_dir, all2mem=True):
        """
        All images from `img_dir` will be read.
        """
        self.img_dir = img_dir
        self.all2mem = all2mem
        self.img_names = [file for file in os.listdir(self.img_dir)]
        
        assert all([img.endswith('.jpg') for img in self.img_names]), "Must be *.jpg"
        
        if self.all2mem:
            self.images = [io.imread(os.path.join(self.img_dir, img)) 
                           for img in self.img_names]
        
    
    def __len__(self):
        return len(self.img_names)
    
   
    
    def __getitem__(self, idx):
        """
        Get an image in Lab color space.
        Returns a tuple (L, ab, name)
            - `L` stands for lightness - it's the net input
            - `ab` is chrominance - something that the net learns
            - `name` - image filename
        """
        
        if self.all2mem:
            image = self.images[idx]
        else:
            img_name = os.path.join(self.img_dir, self.img_names[idx])
            image = io.imread(img_name)
        
        
        assert image.shape == (224, 224, 3)
                
        img_lab = color.rgb2lab(image)
        img_lab = np.transpose(img_lab, (2, 0, 1))
        
        assert img_lab.shape == (3, 224, 224)
        
        img_lab = torch.tensor( img_lab.astype(np.float32) )
        
        assert img_lab.shape == (3, 224, 224)
               
        L  = img_lab[:1,:,:]
        ab = img_lab[1:,:,:]
        
        # Normalization
        L =   L / 100.0         # 0..1
        ab = (ab + 128.0) / 255.0 # 0..1
              
        assert L.shape == (1, 224, 224)
        assert ab.shape == (2, 224, 224)
        
        return L, ab, self.img_names[idx]   

### Load data

In [ ]:
trainset = ImagesDateset(img_dir_train, all2mem=True)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE,
                                          shuffle=False, num_workers=4)

testset = ImagesDateset(img_dir_test, all2mem=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE,
                                          shuffle=False, num_workers=4)

devset = ImagesDateset(img_dir_dev, all2mem=True)
devloader = torch.utils.data.DataLoader(devset, batch_size=BATCH_SIZE,
                                        shuffle=False, num_workers=4)

### The Network

In [ ]:
class ColNet(nn.Module):
    def __init__(self):

        super(ColNet, self).__init__()
        
        ksize = np.array( [1, 64, 128, 128, 256, 256, 512, 512, 256, 128, 64, 64, 32] ) // 8
        ksize[0] = 1
        
        # 'Low-level features'
        self.conv1 = nn.Conv2d(in_channels=1,        out_channels=ksize[1], kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(in_channels=ksize[1], out_channels=ksize[2], kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=ksize[2], out_channels=ksize[3], kernel_size=3, stride=2, padding=1)
        self.conv4 = nn.Conv2d(in_channels=ksize[3], out_channels=ksize[4], kernel_size=3, stride=1, padding=1)
        self.conv5 = nn.Conv2d(in_channels=ksize[4], out_channels=ksize[5], kernel_size=3, stride=2, padding=1)
        self.conv6 = nn.Conv2d(in_channels=ksize[5], out_channels=ksize[6], kernel_size=3, stride=1, padding=1)
        
        # 'Mid-level fetures'
        self.conv7 = nn.Conv2d(in_channels=ksize[6], out_channels=ksize[7], kernel_size=3, stride=1, padding=1)
        self.conv8 = nn.Conv2d(in_channels=ksize[7], out_channels=ksize[8], kernel_size=3, stride=1, padding=1)
        
        # 'Colorization network'
        self.conv9 = nn.Conv2d(in_channels=ksize[8], out_channels=ksize[9], kernel_size=3, stride=1, padding=1)
        
        # Here comes upsample #1
        
        self.conv10 = nn.Conv2d(in_channels=ksize[9], out_channels=ksize[10], kernel_size=3, stride=1, padding=1)
        self.conv11 = nn.Conv2d(in_channels=ksize[10],out_channels=ksize[11], kernel_size=3, stride=1, padding=1)
        
        # Here comes upsample #2        
        
        self.conv12 = nn.Conv2d(in_channels=ksize[11], out_channels=ksize[12], kernel_size=3, stride=1, padding=1)
        self.conv13 = nn.Conv2d(in_channels=ksize[12], out_channels=2, kernel_size=3, stride=1, padding=1)
        
        
    def forward(self, x):
        
        # Low level
        out = F.relu(self.conv1(x))
        out = F.relu(self.conv2(out))
        out = F.relu(self.conv3(out))
        out = F.relu(self.conv4(out))
        out = F.relu(self.conv5(out))
        out = F.relu(self.conv6(out))
        
        # y = out
        # z = out
        # y → mid level
        # z → global features 
        
        # Mid level
    
        out = F.relu(self.conv7(out))
        out = F.relu(self.conv8(out))
        
        # assert out.shape[1:] == (256, 28, 28), "おわり： mid level"
        

        # Colorization Net
        out = F.relu(self.conv9(out))
        
        # assert out.shape[1:] == (128, 28, 28), "おわり： conv9"
        
        out = nn.functional.interpolate(input=out, scale_factor=2, mode='nearest')

        # assert out.shape[1:] == (128, 56, 56), "おわり： upsample1"
    
        out = F.relu(self.conv10(out))
        out = F.relu(self.conv11(out))
        
        out = nn.functional.interpolate(input=out, scale_factor=2, mode='nearest')


        out = F.relu(self.conv12(out))
        out = torch.sigmoid(self.conv13(out))
        
        out = nn.functional.interpolate(input=out, scale_factor=2, mode='nearest')
        
        # assert out.shape[1:] == (2, 224, 224)
        
        return out
        

### Utilities

Unnormalize and return RGB image

In [ ]:
def net_out2rgb(L, ab_out):
    """
    L - original `L` channel
    ab_out - learned `ab` channels which were the net's output
    
    Retruns: 3 channel RGB image
    """
    # Convert to numpy and unnnormalize
    L = L.numpy() * 100.0
    
    ab_out = np.floor(ab_out.numpy() * 255.0) - 128.0 
    
    # Transpose axis to HxWxC again
    L = L.transpose((1, 2, 0))
    ab_out = ab_out.transpose((1, 2, 0))

    # Stack layers
    lab_stack = np.dstack((L, ab_out))
    
    return color.lab2rgb(lab_stack)

### Training

In [ ]:
net = ColNet()
mse = nn.MSELoss(reduction='sum')
optimizer = optim.Adam(net.parameters(), lr=0.0001)

model_filename = ""

loss_history = []

for epoch in range(EPOCHS): 

    epoch_loss = 0.0
    dev_loss = 0.0
    
    ################################ 
    ##   TRAINING   ################
    ################################

    print("Epoch {} / {}".format(epoch + 1, EPOCHS))
    
    # Turn train mode on
    net.train() 
    for batch_idx, train_data in enumerate(trainloader):

        L, ab, _ = train_data
        
        optimizer.zero_grad()
        ab_out = net(L)
        
        
        assert ab.shape == ab_out.shape
        
        loss = mse(ab, ab_out)      
        
#         myloss_avg = ((ab - ab_out)**2).mean().item()
#         myloss_sum = ((ab - ab_out)**2).sum().item()
#         print("loss: {:.4f}  \t myloss_avg: {:.4f} \t myloss_sum {:.4f}"
#               .format(loss, myloss_avg, myloss_sum))
        
        loss.backward()
        optimizer.step()
        
       

        batch_loss = loss.item()
        
        print('[{} / {}] batch loss: {:.3f}'
            .format(batch_idx + 1, len(trainloader), batch_loss))
        epoch_loss += batch_loss
    
    #
    # len(trainloader) - ilość batch
    # len(trainloader.dataset) - ilość zdjęć
    # Uwaga: ilość_batch * batch_size != ilość zdjęć 
    # (bo ostatni batch może mieć mniej zdjęć, 
    # gdy batch_size nie dzieli #zdj)
    # 
    
    print("\n")
    print('SUM of all batch losses in epoch: {:.2f}'.format(epoch_loss))
    print('MEAN over all batches in epoch: {:.2f}'.format(epoch_loss/len(trainloader)))
    
    epoch_loss /= len(trainloader)
    
    #################################
    ##  VALIDATE on dev set  ########
    #################################
    print("\nValidating...")
    
    # Turn eval mode on
    net.eval()
    with torch.no_grad():
        
        dev_loss = 0.0
        
        for batch_idx, dev_data in enumerate(devloader):

            L_dev, ab_dev, _ = dev_data

            ab_dev_output = net(L_dev)

            assert ab_dev.shape == ab_dev_output.shape
            
            dev_batch_loss = mse(ab_dev_output, ab_dev)
            
#             mydev_loss_sum = ((ab_dev - ab_dev_output)**2).sum().item()
#             mydev_loss_avg = ((ab_dev - ab_dev_output)**2).mean().item()            
#             print("dev_loss {} \t  myloss_avg {} \t myloss_sum {}"
#                   .format(dev_loss, mydev_loss_avg, mydev_loss_sum))

            dev_loss += dev_batch_loss
            print("[{} / {}] dev batch loss: {}"
                  .format(batch_idx+1, len(devloader), dev_batch_loss))
            
            
            
        print("sum of dev losses {}".format(dev_loss.item()))
        print("mean of dev losses {}".format(dev_loss.item()/len(devloader)))
        
        
        loss_history.append((epoch_loss, dev_loss/len(devloader)))

            
    # Save
    # model_filename = './model/colnet{}.pt'.format(time.strftime("%y%m%d-%H-%M-%S"))
    model_filename = './model/colnet.pt'
    torch.save(net.state_dict(), model_filename)
    print('\nsaved model to {}'.format(model_filename))
    
    print('~ end of epoch {}\n\n\n'.format(epoch + 1))

print('Finished Training')

In [ ]:
hist_epoch = [h[0] for h in loss_history]
hist_dev   = [h[1] for h in loss_history]
Xs = np.arange(0, EPOCHS, 1)
plt.plot(Xs, hist_epoch, label='Epoch loss')
plt.plot(Xs, hist_dev, label='Dev loss')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend(loc='upper right');
# plt.ylim(1500, 2200)
plt.show()

### Load and test model

In [ ]:
!rm -rf out/*

In [ ]:
# model_filename = "./model/colnet181105-21-36-20.pt"
model_filename = "./model/colnet.pt"

print("Make sure you're using up to date model!!!")    
print("Colorizing {} using {}\n".format(img_dir_test, model_filename))


net = ColNet()
net.load_state_dict(torch.load(model_filename))
net.eval()

with torch.no_grad():
    for batch_no, data in enumerate(testloader):
        
        print("Processing batch {} / {}".format(batch_no + 1, len(testloader)))
        L, ab, name = data
        ab_outputs = net(L)
        
        for i in range(L.shape[0]):
            img = net_out2rgb(L[i], ab_outputs[i])
            io.imsave(os.path.join("./out/", name[i]), img)

print("Saved all photos to ./out/")